In [8]:
# import drf

# drf.hello(1, 2)

4

In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro

import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects import pandas2ri
pandas2ri.activate()
#from rpy2 import robjects
#from rpy2.robjects.vectors import IntVector
#rpy2.robjects.NameClassMap = rpy2.robjects.conversion.NameClassMap
#from rpy2_Matrix import Matrix
#from rpy2_Matrix import conversion

# Choosing a CRAN Mirror
#import rpy2.robjects.packages as rpackages
#utils = rpackages.importr('utils')
#utils.chooseCRANmirror(ind=1)

# Installing required packages
#from rpy2.robjects.vectors import StrVector
#packages = ('drf')
#utils.install_packages(StrVector(packages))

# Import packages
from rpy2.robjects.packages import importr
base, drf_r_package = importr('base'), importr('drf')

# Import Functions
#fit, predict = drf.drf, drf.predict_drf

def convert_to_R(X):
    if type(X) == np.ndarray:
        if len(X.shape) == 1: 
            X = X.reshape(1, X.shape[0])
        nr, nc = X.shape
        return ro.r.matrix(X, nrow=nr, ncol=nc)

    if type(X) == pd.core.frame.DataFrame:
        #tmp = X.to_numpy()
        #nr, nc = tmp.shape
        #return ro.r.matrix(tmp, nrow=nr, ncol=nc)
        return ro.conversion.py2rpy(X)

class drf:
    def __init__(self, **fit_params):
        self.fit_params = fit_params 
    
    class predict_output: pass
    
    def fit(self, X, Y):
        self.X_train = X
        self.Y_train = Y
        
        X_r = convert_to_R(X)
        Y_r = convert_to_R(Y)
        
        self.r_fit_object = drf_r_package.drf(X_r, Y_r, **self.fit_params)

    def predict(self, newdata, **predict_params):
        newdata_r = convert_to_R(newdata)
        r_output = drf_r_package.predict_drf(self.r_fit_object, newdata_r)
        
        weights = base.as_matrix(r_output[0])
    
        #y = r_output[1]
        ret = self.predict_output()
        ret.weights = weights
        ret.y = r_output[1]
        return ret
        #Ytr = self.Y_train#.apply(transformation)
        
        if functional=='weights':
            ret.weights = weights
        #    ret.y = Ytr
        elif functional=='mean':
            ret.mean = np.zeros((newdata.shape[0], Ytr.shape[1]))
            for i in range(newdata.shape[0]):
                ret.mean[i,:] = Ytr.multiply(weights[i,:], axis=0).mean().to_numpy() 
        elif functional=='sd':
            pass
        elif functional=='quantile':
            pass
        elif functional=='cov':
            pass
        elif functional=='cor':
            pass
        return ret  
                    

/Users/cevidd/opt/anaconda3/lib/python3.7/site-packages/rpy2/robjects/vectors.py:969: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


In [2]:
import numpy as np
import pandas as pd

n=1000
p=10
d=2
X = np.random.normal(0, 1, size=(n, p))
Y = np.random.normal(0, 1, size=(n, d))
Y[:,0] = Y[:,0] + X[:,0]
Y[:,1] = Y[:,1] * X[:,1]

#X = pd.DataFrame(X)
#Y = pd.DataFrame(Y)

In [3]:
regr = drf(mtry=3, bandwidth=1.1, min_node_size=10, num_trees=100, num_features=10, splitting_rule="CART")
regr.fit(X, Y)
#out = regr.predict(newdata=X)#, transformation=lambda y : y)#, quantiles=np.array([0.1, 0.5, 0.9]))
#fix quantiles parameter to be of any type
#reformat quantiles output
#out.weights